<a href="https://colab.research.google.com/github/shaunak09vb/CNN-Model-KN/blob/main/CNN_KN_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Keras Tuner

In [ ]:
!pip install keras-tuner

# Importing Libraries

In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters 

In [4]:
print(tf.__version__)

2.3.0


# Importing Data

In [5]:
fashion_mnist=keras.datasets.fashion_mnist

In [6]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


# Data Wrangling

### Converting Images from Grey-scale to Color Images

In [7]:
train_images=train_images/255.0
test_images=test_images/255.0

In [8]:
train_images[0].shape

(28, 28)

In [9]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [12]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [13]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [14]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 19s]
val_accuracy: 0.9110000133514404

Best val_accuracy So Far: 0.9110000133514404
Total elapsed time: 00h 02m 00s
INFO:tensorflow:Oracle triggered exit


In [15]:
model_para=tuner_search.get_best_models(num_models=1)[0]

In [16]:
model_para.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 48)        480       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        27712     
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                1769520   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                490       
Total params: 1,798,202
Trainable params: 1,798,202
Non-trainable params: 0
_________________________________________________________________


In [17]:
model_para.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1273 - accuracy: 0.9532 - val_loss: 0.2699 - val_accuracy: 0.9143
Epoch 5/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0907 - accuracy: 0.9664 - val_loss: 0.2789 - val_accuracy: 0.9137
Epoch 6/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0624 - accuracy: 0.9769 - val_loss: 0.3420 - val_accuracy: 0.9138
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0445 - accuracy: 0.9838 - val_loss: 0.3656 - val_accuracy: 0.9173
Epoch 8/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0353 - accuracy: 0.9876 - val_loss: 0.4405 - val_accuracy: 0.9133
Epoch 9/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0309 - accuracy: 0.9891 - val_loss: 0.4932 - val_accuracy: 0.9138
Epoch 10/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0213 - accuracy: 0.9925 - val_loss: 0.5160 - val_accuracy